In [1]:
%load_ext autoreload
%autoreload 2

# Explaining why so many circuits have a sensitivity of 0.000333

In [ ]:
import numpy as np
import os
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from common import set_theme

set_theme()

top_write_dir = os.path.join('data', '26_sensitivity_peak2')

In [3]:
dir_sim = os.path.join('..', 'notebooks', 'data', 'simulate_circuits', '2025_09_23__15_41_51')

fn_analytics = os.path.join(dir_sim, 'analytics.json')
with open(fn_analytics, 'r') as f:
    analytics = json.load(f)
    
fn_circuits = '../data/raw/generate_sequence_batch/2025_09_20_103744/circuits/rc.json'
fn_energies = '../data/raw/generate_sequence_batch/2025_09_20_103744/energies.json'

with open(fn_circuits, 'r') as f:
    circuits = json.load(f)
with open(fn_energies, 'r') as f:
    energies = json.load(f)
    
cols_e = [f'$RNA_{i+1}$-$RNA_{j+1}$' for i, j in zip(*np.triu_indices(3))]
energy_keys = sorted(energies.keys(), key=lambda k: int(k.split('_')[1]))


In [4]:
set_theme()
